In [ ]:
!pip install crewai==0.30.11 crewai-tools==0.1.6 langchain-groq requests pyyaml -q

In [ ]:
!pip install scikit-learn pandas

In [ ]:
import os
import yaml
import requests
from google.colab import userdata
from crewai import Agent, Task, Crew, Process
from langchain_groq import ChatGroq


os.environ["GROQ_API_KEY"] = userdata.get('GEEMA')

manager_llm = ChatGroq(
    temperature=0.2,
    model_name="deepseek-r1-distill-llama-70b"
)

agent_llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant"
)

In [ ]:
import pandas as pd

path = "/content/sample_data/complex_english_dataset - Sheet1.csv"

col_names = ['text','grd_truth']

df = pd.read_csv(path, header=None, names=col_names)

texts = df['text']



In [ ]:
mode_agent_a1 = Agent(
    role = "Linguistic Analyst specializing in Modality and Conditionals.",
    goal = """Identify any non-factual or hypothetical statements (irrealis mood) in the text.
   **Your final output MUST be a single, concise phrase** that includes your analysis and confidence score.""",
    backstory = """You are a meticulous, logic-driven linguist who believes sentiment can only be assessed from concrete facts.
    Your analysis MUST strictly focus on identifying words of possibility or speculation (e.g., "could," "may," "if," "plans to").
    **Your confidence score should be high (7-10) ONLY if you find clear instances of such language**.
    If the text is purely factual and literal, your analysis is less critical, so your analysis_text MUST be one sentence and assign a low confidence score (1-4).""",
    llm = agent_llm,
    verbose = True,
    allow_delegation=False,
    max_iter = 1
)

rhetoric_agent_a2 = Agent(
    role = " Financial Disinformation and Rhetoric Specialist.",
    goal = """Detect and decipher any rhetorical devices (sarcasm, hyperbole) to uncover the true sentiment. **Your final output MUST be a single, concise phrase** that includes your analysis and confidence score.""",
    backstory = """You are a cynical ex-financial journalist who reads between the lines. You MUST identify the specific
    rhetorical device and explain how it alters the literal meaning. **Assign a high confidence score (7-10) ONLY if you
    detect clear rhetoric**. If the text is straightforward, your unique skill is not needed, so your analysis_text MUST be one sentence and report a low
    confidence score (1-4).""",
    llm = agent_llm,
    verbose = True,
    allow_delegation=False,
    max_iter = 1
)



comparative_logic_agent3 = Agent(
    role="Comparative Financial Analyst",
    goal="To identify any comparative statements in the text, determine the sentiment of the comparison (which entity is favored), and correctly attribute the sentiment to each entity involved. **Your final output MUST be a single, concise phrase** that includes your analysis and confidence score.",
    backstory="""You are an expert in logical reasoning and linguistics, specializing in comparative structures. Your first step is to scan for keywords of comparison (e.g., 'better than', 'outperformed', 'in contrast to'). Once a comparison is found, you must identify the entities being compared and determine the sentiment direction for each one separately.

    **Example:**
    * **Input Text:** "Company A's performance was stronger than Company B's."
    * **Your Correct Output:** "A direct comparison was found where Company A is viewed positively and Company B is viewed negatively. My confidence is 9."

    If no comparison is detected, you must state that and assign a low confidence score.
    """,
    llm=agent_llm,
    verbose=True,
    allow_delegation=False,
    max_iter=1
)


reference_agent_a5 = Agent(
    role = "Contextual Intelligence Analyst.",
    goal = """Analyze the text for references to external facts (prices, dates, events) and explain how this context
    influences the sentiment. **Your final output MUST be a single, concise phrase** that includes your analysis and confidence score. """,
    backstory = """You are a seasoned analyst with a memory of market history. **Your confidence score should be high (7-10)
    ONLY if understanding the sentiment truly depends on knowing about an external event or fact mentioned in the text**.
    If the text is self-contained and requires no external knowledge, your analysis_text MUST be one sentence and your confidence MUST be low (1-4).""",
    llm = agent_llm,
    verbose = True,
    allow_delegation=False,
    max_iter = 1
)

institutional_agent_a6 = Agent(
    role = "Long-Term Value Investment Analyst.",
    goal = """Analyze the text from a cautious institutional investor's perspective, focusing on long-term fundamentals.
   **Your final output MUST be a single, concise phrase** that includes your analysis and confidence score.""",
    backstory = """You are a professional analyst from a top-tier investment firm. You disregard short-term market hype.
    **Your confidence score should be high (7-10) if the text discusses fundamentals like management strategy, profitability,
    debt, or industry trends**. If the text is purely about short-term price speculation, it is outside your focus, so your analysis_text MUST be one sentence and you MUST
    assign a low confidence score (1-4).""",
    llm = agent_llm,
    verbose = True,
    allow_delegation=False,
    max_iter = 1
)

individual_agent_a7 = Agent(
    role = "Retail Momentum Trader.",
    goal = """ Analyze the text from a short-term individual investor's perspective. **Your final output MUST be a single, concise phrase** that includes your analysis and confidence score.""",
    backstory = """ You are a retail trader influenced by social media hype and immediate price action. You IGNORE long-term
    fundamentals. **Your confidence score should be high (7-10) ONLY if the text is filled with market buzz, price targets,
    or informal social media language**. If the text is a formal, fundamental analysis, it is not relevant to your trading style, so your analysis_text MUST be
     one sentence,
    so your confidence MUST be low (1-4).""",
    llm = agent_llm,
    verbose = True,
    allow_delegation=False,
    max_iter = 1
)

sentiment_agent_a8 = Agent(
    role="Final Sentiment Synthesizer",
    goal="To analyze a collection of specialist reports and produce a single, final sentiment classification (Positive, Negative, or Neutral) in a detailed JSON format.",
    backstory="""You are a final analysis agent. Your sole purpose is to synthesize the findings of a specialist team, which have been provided to you, into a definitive sentiment judgment with clear, transparent reasoning.

    **Your Process:**
    1.  **Review Inputs:** Carefully review all the specialist analyses provided to you.
    2.  **Prioritization Logic:** Your decision must be driven by the confidence scores included in each analysis. Prioritize high-confidence reports (7-10) as your primary evidence.
    3.  **Conflict Reporting:** If high-confidence reports conflict, you must clearly report the nature of the conflict in the 'conflicting_signals' field.
    4.  **Final Output Mandate:** Your final response MUST be ONLY the single, clean JSON object with the five required keys. Do not add any conversational text or other actions.

    **CRITICAL RULE:** You do not have the ability to delegate tasks to other agents. You must form your final conclusion based **only** on the analyses already provided to you. Your only permitted action is to provide the "Final Answer" in the specified JSON format.
    """,
    llm=manager_llm,
    verbose=True,
    allow_delegation=False,
    max_iter = 5
)

text_to_analyze = "{english_text}"

task_a1 = Task(
    description=f"Analyze the following text for its mood and modality. Text: {text_to_analyze}",
    expected_output="A short phrase summarizing the modality analysis and confidence. Example: 'Hypothetical statement detected, confidence 8'",
    agent=mode_agent_a1
)

task_a2 = Task(
    description=f"Analyze the following text for rhetorical devices. Text: {text_to_analyze}",
    expected_output="A short phrase summarizing the rhetorical analysis and confidence. Example: 'Sarcasm detected, confidence 9'",
    agent=rhetoric_agent_a2
)

task_a3 = Task(
    description=f"Analyze the text for any comparative statements: {text_to_analyze}",
    expected_output="A phrase summarizing the comparative analysis and confidence. Example: 'Direct comparison found: Entity A (Positive) vs Entity B (Negative), confidence 9'",
    agent=comparative_logic_agent3
)

task_a5 = Task(
    description=f"Analyze the following text for external references. Text: {text_to_analyze}",
    expected_output="A phrase summarizing the impact of external context and confidence. Example: 'Sentiment influenced by recent market crash, confidence 8'",
    agent=reference_agent_a5
)

task_a6 = Task(
    description=f"Analyze the following text from an institutional investor's perspective. Text: {text_to_analyze}",
    expected_output="A phrase summarizing the fundamental analysis and confidence. Example: 'Positive long-term fundamentals, confidence 7'",
    agent=institutional_agent_a6
)

task_a7 = Task(
    description=f"Analyze the following text from an individual investor's perspective. Text: {text_to_analyze}",
    expected_output="A phrase summarizing the short-term market sentiment and confidence. Example: 'Strong bullish momentum detected, confidence 9'",
    agent=individual_agent_a7
)

sentiment_agents_task = Task(
    description = """
     Synthesize the analyses from all specialist agents into a final sentiment judgment. Weigh each analysis according to its confidence
     score as per your instructions.
    """,
    expected_output = """ "The final, synthesized report in a single JSON object with the five required keys: "
        "'final_sentiment', 'overall_confidence', 'justification', 'key_contributing_agents', and 'conflicting_signals'." """,
    agent = sentiment_agent_a8,
    context=[
        task_a1,
        task_a2,
        task_a3,
        task_a5,
        task_a6,
        task_a7
    ]
)


crew = Crew(
    agents=[
        mode_agent_a1,
        rhetoric_agent_a2,
        comparative_logic_agent3,
        reference_agent_a5,
        institutional_agent_a6,
        individual_agent_a7,
        sentiment_agent_a8
    ],
    tasks=[
        task_a1,
        task_a2,
        task_a3,
        task_a5,
        task_a6,
        task_a7,
        sentiment_agents_task
    ],
    process=Process.sequential,
    verbose = 2
)



results = []

for text in texts:
  result = crew.kickoff(inputs={'english_text': text})
  results.append(result)





In [ ]:
import json
import re

for index, result_string in enumerate(results):

  match = re.search(r'\{.*\}', result_string, re.DOTALL)
  if match:
    json_part = match.group(0)
    try:
      data_dict = json.loads(json_part)
      print(
          f"\n----------- Result {index + 1} -----------"
          f"\nSentiment: {data_dict.get('final_sentiment', 'N/A')}"
          f"\nConfidence: {data_dict.get('overall_confidence', 'N/A')}"
          f"\nJustification: {data_dict.get('justification', 'N/A')}"
          f"\nKey Agents: {data_dict.get('key_contributing_agents', 'N/A')}"
          f"\nConflicts: {data_dict.get('conflicting_signals', 'N/A')}"
      )
    except json.JSONDecodeError as e:
      print(f"\n----------- Result {index + 1} (JSON Parse Error) -----------")
      print(f"Could not parse extracted JSON: {e}")
      print(f"Extracted JSON part: {json_part}")
  else:
    print(f"\n----------- Result {index + 1} (No JSON Found) -----------")
    print(f"Could not find JSON object in the result string.")
    print(f"Raw result string: {result_string}")


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
import json
import re

pred = []

for result_string in results:
  match = re.search(r'\{.*\}', result_string, re.DOTALL)
  if match:
    json_part = match.group(0)
    try:
      data_dict = json.loads(json_part)
      pred.append(data_dict.get('final_sentiment', 'ERROR').lower())
    except json.JSONDecodeError:
      pred.append('ERROR')
  else:
    pred.append('ERROR')
ground_truth = df['grd_truth'].str.lower().str.strip().tolist()


min_len = min(len(ground_truth), len(pred))
ground_truth = ground_truth[:min_len]
pred = pred[:min_len]


accurecy = accuracy_score(ground_truth, pred)
f1_score_metric = f1_score(ground_truth, pred, average='weighted')
print(f"Accuracy: {accurecy: .4f}    F1 score: {f1_score_metric: .4f}")
print(classification_report(ground_truth, pred))